In [ ]:
!pip install wandb

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
wandb.init(project="Cifar10", entity="gharabadiyan")

wandb: Currently logged in as: gharabadiyan. Use `wandb login --relogin` to force relogin


In [6]:
cifar10= tf.keras.datasets.cifar10

(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

X_test = X_test / 255.0

X_train,X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=24)

X_train = X_train / 255.0

170508288/170498071 [==============================] - 2s 0us/step


## MLP

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32,3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=15,validation_data=(X_val,Y_val),callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/15
1250/1250 [==============================] - 7s 3ms/step - loss: 1.9279 - accuracy: 0.3141 - val_loss: 182.4086 - val_accuracy: 0.3364 - _timestamp: 1654669435.0000 - _runtime: 106.0000
Epoch 2/15
1250/1250 [==============================] - 5s 4ms/step - loss: 1.7816 - accuracy: 0.3638 - val_loss: 189.0929 - val_accuracy: 0.3610 - _timestamp: 1654669440.0000 - _runtime: 111.0000
Epoch 3/15
1250/1250 [==============================] - 3s 3ms/step - loss: 1.7218 - accuracy: 0.3872 - val_loss: 169.3339 - val_accuracy: 0.3842 - _timestamp: 1654669443.0000 - _runtime: 114.0000
Epoch 4/15
1250/1250 [==============================] - 4s 3ms/step - loss: 1.6869 - accuracy: 0.3982 - val_loss: 182.2294 - val_accuracy: 0.3932 - _timestamp: 1654669447.0000 - _runtime: 118.0000
Epoch 5/15
1250/1250 [==============================] - 4s 3ms/step - loss: 1.6592 - accuracy: 0.4104 - val_loss: 222.3869 - val_accuracy: 0.3658 - _timestamp: 1654669450.0000 - _runtime: 121.0000
Epoch 6/15
1250

In [8]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.5996 - accuracy: 0.4268


[1.5995559692382812, 0.426800012588501]

In [9]:
model.save('MLP_Cifar10.h5')

## MLP + CNN

In [10]:
model = tf.keras.models.Sequential([
                                    
        layers.Conv2D(32,(3,3),activation='relu',input_shape=(32, 32,3)),
        layers.MaxPool2D((2, 2)),
        layers.Conv2D(64,(3,3),activation='relu'),                            
        layers.MaxPool2D((2, 2)),
        layers.Conv2D(64,(3,3),activation='relu'),

        layers.Flatten(),

        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
model.fit(X_train,Y_train,epochs=10,validation_data=(X_val,Y_val),callbacks=[WandbCallback()])

Epoch 1/10
1250/1250 [==============================] - 14s 4ms/step - loss: 1.5704 - accuracy: 0.4239 - val_loss: 234.0733 - val_accuracy: 0.3600 - _timestamp: 1654669613.0000 - _runtime: 284.0000
Epoch 2/10
1250/1250 [==============================] - 6s 5ms/step - loss: 1.2178 - accuracy: 0.5688 - val_loss: 150.9868 - val_accuracy: 0.4729 - _timestamp: 1654669619.0000 - _runtime: 290.0000
Epoch 3/10
1250/1250 [==============================] - 5s 4ms/step - loss: 1.0746 - accuracy: 0.6224 - val_loss: 212.7194 - val_accuracy: 0.4330 - _timestamp: 1654669624.0000 - _runtime: 295.0000
Epoch 4/10
1250/1250 [==============================] - 6s 4ms/step - loss: 0.9681 - accuracy: 0.6601 - val_loss: 233.2188 - val_accuracy: 0.4375 - _timestamp: 1654669629.0000 - _runtime: 300.0000
Epoch 5/10
1250/1250 [==============================] - 5s 4ms/step - loss: 0.8929 - accuracy: 0.6858 - val_loss: 212.2615 - val_accuracy: 0.4284 - _timestamp: 1654669634.0000 - _runtime: 305.0000
Epoch 6/10
125

In [12]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.9205 - accuracy: 0.6988


[0.9204718470573425, 0.6988000273704529]

In [13]:
model.save('MLP_CNN_Cifar10.h5')